In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import os


config_file = "/home/zjusst/qms/webarena/config_files/612.json"    # Choose the corresponding config file based on the website to be sampled
config_randomwalk = "/home/zjusst/qms/webarena/my_scripts/stage_1_explore/config_randomwalk"
traj_dir = "/home/zjusst/qms/webarena/my_scripts/stage_1_explore/traj_results"
screen_dir = "/home/zjusst/qms/webarena/my_scripts/stage_1_explore/screen_results"
element_dir = "/home/zjusst/qms/webarena/my_scripts/stage_1_explore/click_elements"


if not os.path.exists(traj_dir):
    os.mkdir(traj_dir)
if not os.path.exists(screen_dir):
    os.mkdir(screen_dir)
if not os.path.exists(element_dir):
    os.mkdir(element_dir)
if not os.path.exists(config_randomwalk):
    os.mkdir(config_randomwalk)

SLEEP = 3


os.environ[
    "SHOPPING"
] = "http://10.130.138.30:7770"
os.environ[
    "SHOPPING_ADMIN"
] = "http://10.130.138.30:7780/admin"
os.environ[
    "REDDIT"
] = "http://10.130.138.30:9999"
os.environ[
    "GITLAB"
] = "http://10.130.138.30:8023"
os.environ[
    "MAP"
] = "http://10.130.138.30:3000"
os.environ[
    "WIKIPEDIA"
] = "http://10.130.138.30:8888/wikipedia_en_all_maxi_2022-05/A/User:The_other_Kiwix_guy/Landing"
os.environ[
    "HOMEPAGE"
] = "PASS"  # The home page is not currently hosted in the demo site
print("Done setting up URLs")

Done setting up URLs


In [3]:
# 选择随机 url 和新建 config 文件

import json
import random
import uuid

# Check if configuration file is correct
assert os.path.exists(config_file)
with open(config_file, "r") as f:
    config = json.load(f)

# Check which URLs are available for the corresponding website and randomly select one
# configs_dir = "/home/zjusst/qms/webarena/config_files"
# website_url = dict()
# for config_item in os.listdir(configs_dir):
#     if config_item == "examples" or "test" in config_item:
#         continue
#     config_path = os.path.join(configs_dir, config_item)
#     config_item = json.load(open(config_path, 'r'))
#     if not isinstance(config_item, dict):
#         continue
#     try:
#         if len(config_item["sites"]) != 1:
#             continue
#         website_name = config_item["sites"][0]
#         if website_name not in website_url:
#             website_url[website_name] = set()

#         start_url = config_item["start_url"]
#         website_url[website_name].add(start_url)
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         print(config_item)

web_urls = ['http://10.130.138.30:9999',
 'http://10.130.138.30:9999/f/pics',
 'http://10.130.138.30:9999/f/pittsburgh/45899/driving-in-pittsburgh-summed-up-by-one-traffic-sign',
 'http://10.130.138.30:9999/f/singularity/69404/this-is-how-chatgpt-sees-itself',
 'http://10.130.138.30:9999/f/technology/134852/ai-experts-disown-musk-backed-campaign-citing-their-research',
 'http://10.130.138.30:9999/f/books/59421/friendly-reminder-bookshop-org-exists']
random_url = random.choice(list(web_urls))
print(f"Random URL: {random_url}")
config["start_url"] = random_url
# Save config
unique_id_traj = str(uuid.uuid4())
config_randomwalk_path = os.path.join(config_randomwalk, f"{unique_id_traj}.json")
json.dump(config, open(config_randomwalk_path, 'w'))

Random URL: http://10.130.138.30:9999/f/technology/134852/ai-experts-disown-musk-backed-campaign-citing-their-research


In [4]:
config

{'sites': ['reddit'],
 'task_id': 612,
 'require_login': True,
 'storage_state': './.auth/reddit_state.json',
 'start_url': 'http://10.130.138.30:9999/f/technology/134852/ai-experts-disown-musk-backed-campaign-citing-their-research',
 'geolocation': None,
 'intent_template': 'Post a review of my recent reading "{{book}}" in the r/books with my comment "{{content}}".',
 'instantiation_dict': {'book': 'big little lies', 'content': "can't stop it"},
 'intent': 'Post a review of my recent reading "big little lies" in the r/books with my comment "can\'t stop it".',
 'require_reset': False,
 'eval': {'eval_types': ['url_match', 'program_html'],
  'reference_answers': None,
  'reference_url': 'http://10.130.138.30:9999/f/books',
  'program_html': [{'url': "func:reddit_get_post_url('__last_url__')",
    'locator': "document.querySelector('.submission__inner').outerText",
    'required_contents': {'must_include': ['big little lies',
      "can't stop it"]}}],
  'url_note': 'GOLD in PRED'},
 'in

In [5]:
"""
# run bash prepare.sh to save all account cookies, this only needs to be done once
subprocess.run(["bash", "prepare.sh"])
print("Done saving account cookies")
"""
def load_element_pool(file_path):
    """Load element pool"""
    if not os.path.exists(file_path):
        return set()
    else:
        with open(file_path, 'r') as f:
            data_list = json.load(f)
            element_pool = set((tuple(item[0]), item[1]) for item in data_list)
            return element_pool

# Init an environment
from browser_env import (
    Action,
    ActionTypes,
    ObservationMetadata,
    ScriptBrowserEnv,
    StateInfo,
    Trajectory,
    action2str,
    create_id_based_action,
    create_stop_action,
)
from evaluation_harness.evaluators import evaluator_router

# maintain a trajectory
trajectory: Trajectory = []

# Maintain element pools
website_name = config["sites"][0]
unclick_elem_pool_path = os.path.join(element_dir, website_name + "_unclick.json")
click_elem_pool_path = os.path.join(element_dir, website_name + "_click.json")
explored_elem_pool_path = os.path.join(element_dir, website_name + "_explored.json")

unclick_elem_pool = load_element_pool(unclick_elem_pool_path)
click_elem_pool = load_element_pool(click_elem_pool_path)
explored_elem_pool = load_element_pool(explored_elem_pool_path)

env = None

/home/zjusst/anaconda3/envs/webarena/lib/python3.10/site-packages/beartype/_util/hint/pep/utilpeptest.py:345: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Mapping[str, gymnasium.spaces.space.Space[typing.Any]] deprecated by PEP 585 scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". See this discussion for further details and alternatives:
    https://github.com/beartype/beartype#pep-585-deprecations
  warn(


In [6]:
env = ScriptBrowserEnv(
    headless=False,
    slow_mo=100,
    observation_type="accessibility_tree",
    current_viewport_only=True,
    viewport_size={"width": 1280, "height": 720},
)

env.reset(options={"config_file": config_randomwalk_path})


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [2]:
from browser_env.utils import (
    DetachedPage,
)
a = {'page': DetachedPage(url='http://10.130.138.30:9999/f/pics', content=''), 'fail_error': '', 'observation_metadata': {'text': {'obs_nodes_info': {'1': {'backend_id': 2, 'union_bound': [0.0, 0.0, 10.0, 10.0], 'text': "[1] RootWebArea 'pics' focused: True"}, '211': {'backend_id': 145, 'union_bound': [104, 0, 90, 52], 'text': "[211] HeaderAsNonLandmark ''"}, '212': {'backend_id': 147, 'union_bound': [104, 0, 90, 52], 'text': "[212] link 'Home'"}, '244': {'backend_id': 155, 'union_bound': [120, 16, 58, 19], 'text': "[244] StaticText 'Postmill'"}, '245': {'backend_id': 11, 'union_bound': [194, 0, 86, 52], 'text': "[245] link 'Forums'"}, '497': {'backend_id': 170, 'union_bound': [210, 16, 54, 19], 'text': "[497] StaticText 'Forums'"}, '246': {'backend_id': 13, 'union_bound': [280, 0, 62, 52], 'text': "[246] link 'Wiki'"}, '498': {'backend_id': 174, 'union_bound': [296, 16, 30, 19], 'text': "[498] StaticText 'Wiki'"}, '254': {'backend_id': 193, 'union_bound': [462, 7.5, 222, 37], 'text': "[254] searchbox 'Search query'"}, '255': {'backend_id': 18, 'union_bound': [804, 0, 48, 52], 'text': "[255] link 'Notifications (0)'"}, '256': {'backend_id': 20, 'union_bound': [852, 0, 103, 52], 'text': "[256] link 'Submit'"}, '515': {'backend_id': 22, 'union_bound': [888, 16, 51, 19], 'text': "[515] StaticText 'Submit'"}, '18': {'backend_id': 24, 'union_bound': [955, 0, 221, 52], 'text': "[18] button 'MarvelsGrantMan136' hasPopup: menu expanded: False"}, '520': {'backend_id': 26, 'union_bound': [991, 16, 157, 19], 'text': "[520] StaticText 'MarvelsGrantMan136'"}, '19': {'backend_id': 328, 'union_bound': [90, 66, 806, 2380.1875], 'text': "[19] main ''"}, '204': {'backend_id': 330, 'union_bound': [104, 80, 778, 42], 'text': "[204] heading '/f/pics'"}, '226': {'backend_id': 332, 'union_bound': [104, 84, 30, 33], 'text': "[226] StaticText '/f/'"}, '228': {'backend_id': 334, 'union_bound': [134, 84, 53, 33], 'text': "[228] StaticText 'pics'"}, '589': {'backend_id': 342, 'union_bound': [104, 136, 100, 35], 'text': "[589] link 'Submissions'"}, '916': {'backend_id': 343, 'union_bound': [114.5, 145, 79, 16], 'text': "[916] StaticText 'Submissions'"}, '590': {'backend_id': 347, 'union_bound': [204, 136, 89, 35], 'text': "[590] link 'Comments'"}, '917': {'backend_id': 348, 'union_bound': [214.5, 145, 68, 16], 'text': "[917] StaticText 'Comments'"}, '23': {'backend_id': 354, 'union_bound': [307, 136, 72, 35], 'text': "[23] button 'Sort by: Hot' hasPopup: menu expanded: False"}, '921': {'backend_id': 32, 'union_bound': [334.5, 145, 23, 16], 'text': "[921] StaticText 'Hot'"}, '24': {'backend_id': 405, 'union_bound': [104, 185, 778, 69.9375], 'text': "[24] article ''"}, '27': {'backend_id': 410, 'union_bound': [236, 185, 646, 48.9375], 'text': "[27] HeaderAsNonLandmark ''"}, '593': {'backend_id': 413, 'union_bound': [236, 187, 181, 22], 'text': "[593] heading 'A Trejo Thanksgiving.'"}, '942': {'backend_id': 414, 'union_bound': [236, 187, 181, 22], 'text': "[942] link 'A Trejo Thanksgiving.'"}, '944': {'backend_id': 415, 'union_bound': [236, 187, 181, 22], 'text': "[944] StaticText 'A Trejo Thanksgiving.'"}, '945': {'backend_id': 421, 'union_bound': [236, 216.9375, 75, 14], 'text': "[945] StaticText 'Submitted by '"}, '30': {'backend_id': 422, 'union_bound': [311, 216.9375, 63, 14], 'text': "[30] link 'DinoRoman' expanded: False"}, '1482': {'backend_id': 36, 'union_bound': [311, 216.9375, 63, 14], 'text': "[1482] StaticText 'DinoRoman'"}, '946': {'backend_id': 37, 'union_bound': [374, 216.9375, 3, 14], 'text': "[946] StaticText ' '"}, '1484': {'backend_id': 38, 'union_bound': [380, 216.9375, 53, 14], 'text': "[1484] StaticText 't3_z433er'"}, '948': {'backend_id': 39, 'union_bound': [436, 216.9375, 3, 14], 'text': "[948] StaticText ' '"}, '949': {'backend_id': 427, 'union_bound': [439, 216.9375, 63, 14], 'text': "[949] time 'November 24, 2022 at 10:50:35 PM EST'"}, '1488': {'backend_id': 428, 'union_bound': [439, 216.9375, 63, 14], 'text': "[1488] StaticText '3 years ago'"}, '1489': {'backend_id': 439, 'union_bound': [236, 237.9375, 80, 14], 'text': "[1489] link '195 comments'"}, '1735': {'backend_id': 441, 'union_bound': [236, 237.9375, 80, 14], 'text': "[1735] StaticText '195 comments'"}, '264': {'backend_id': 453, 'union_bound': [121, 192, 14, 14], 'text': "[264] button 'Upvote'"}, '596': {'backend_id': 463, 'union_bound': [112, 208, 32, 16], 'text': "[596] StaticText '7522'"}, '267': {'backend_id': 471, 'union_bound': [121, 227, 14, 14], 'text': "[267] button 'Downvote'"}, '31': {'backend_id': 483, 'union_bound': [104, 268.9375, 778, 69.9375], 'text': "[31] article ''"}, '34': {'backend_id': 488, 'union_bound': [236, 268.9375, 646, 48.9375], 'text': "[34] HeaderAsNonLandmark ''"}, '605': {'backend_id': 491, 'union_bound': [236, 270.9375, 424, 22], 'text': "[605] heading 'Amazing shot of a Blue Jay pestering a Bald Eagle'"}, '963': {'backend_id': 492, 'union_bound': [236, 270.9375, 424, 22], 'text': "[963] link 'Amazing shot of a Blue Jay pestering a Bald Eagle'"}, '965': {'backend_id': 493, 'union_bound': [236, 270.9375, 424, 22], 'text': "[965] StaticText 'Amazing shot of a Blue Jay pestering a Bald Eagle'"}, '966': {'backend_id': 499, 'union_bound': [236, 300.875, 75, 14], 'text': "[966] StaticText 'Submitted by '"}, '37': {'backend_id': 500, 'union_bound': [311, 300.875, 75, 14], 'text': "[37] link 'Smash_Factor' expanded: False"}, '1492': {'backend_id': 42, 'union_bound': [311, 300.875, 75, 14], 'text': "[1492] StaticText 'Smash_Factor'"}, '967': {'backend_id': 43, 'union_bound': [386, 300.875, 3, 14], 'text': "[967] StaticText ' '"}, '1494': {'backend_id': 44, 'union_bound': [392, 300.875, 65, 14], 'text': "[1494] StaticText 't3_115q04n'"}, '969': {'backend_id': 45, 'union_bound': [460, 300.875, 3, 14], 'text': "[969] StaticText ' '"}, '970': {'backend_id': 505, 'union_bound': [463, 300.875, 63, 14], 'text': "[970] time 'February 18, 2023 at 3:35:37 PM EST'"}, '1498': {'backend_id': 506, 'union_bound': [463, 300.875, 63, 14], 'text': "[1498] StaticText '3 years ago'"}, '1499': {'backend_id': 517, 'union_bound': [236, 321.875, 80, 14], 'text': "[1499] link '192 comments'"}, '1737': {'backend_id': 519, 'union_bound': [236, 321.875, 80, 14], 'text': "[1737] StaticText '192 comments'"}, '272': {'backend_id': 531, 'union_bound': [121, 275.9375, 14, 14], 'text': "[272] button 'Upvote'"}, '608': {'backend_id': 541, 'union_bound': [112, 291.9375, 32, 16], 'text': "[608] StaticText '7502'"}, '275': {'backend_id': 549, 'union_bound': [121, 310.9375, 14, 14], 'text': "[275] button 'Downvote'"}, '38': {'backend_id': 561, 'union_bound': [104, 352.875, 778, 69.9375], 'text': "[38] article ''"}, '41': {'backend_id': 71, 'union_bound': [236, 352.875, 646, 48.9375], 'text': "[41] HeaderAsNonLandmark ''"}, '617': {'backend_id': 568, 'union_bound': [236, 354.875, 359, 22], 'text': "[617] heading 'We won first place in the costume contest!'"}, '984': {'backend_id': 569, 'union_bound': [236, 354.875, 359, 22], 'text': "[984] link 'We won first place in the costume contest!'"}, '986': {'backend_id': 570, 'union_bound': [236, 354.875, 359, 22], 'text': "[986] StaticText 'We won first place in the costume contest!'"}, '987': {'backend_id': 576, 'union_bound': [236, 384.8125, 75, 14], 'text': "[987] StaticText 'Submitted by '"}, '44': {'backend_id': 577, 'union_bound': [311, 384.8125, 74, 14], 'text': "[44] link 'KrakenDesign' expanded: False"}, '1502': {'backend_id': 48, 'union_bound': [311, 384.8125, 74, 14], 'text': "[1502] StaticText 'KrakenDesign'"}, '988': {'backend_id': 49, 'union_bound': [385, 384.8125, 3, 14], 'text': "[988] StaticText ' '"}, '1504': {'backend_id': 50, 'union_bound': [391, 384.8125, 57, 14], 'text': "[1504] StaticText 't3_yghd0a'"}, '990': {'backend_id': 51, 'union_bound': [451, 384.8125, 3, 14], 'text': "[990] StaticText ' '"}, '991': {'backend_id': 582, 'union_bound': [454, 384.8125, 63, 14], 'text': "[991] time 'October 29, 2022 at 7:36:07 AM EDT'"}, '1508': {'backend_id': 583, 'union_bound': [454, 384.8125, 63, 14], 'text': "[1508] StaticText '3 years ago'"}, '1509': {'backend_id': 594, 'union_bound': [236, 405.8125, 80, 14], 'text': "[1509] link '134 comments'"}, '1739': {'backend_id': 596, 'union_bound': [236, 405.8125, 80, 14], 'text': "[1739] StaticText '134 comments'"}, '280': {'backend_id': 608, 'union_bound': [121, 359.875, 14, 14], 'text': "[280] button 'Upvote'"}, '620': {'backend_id': 618, 'union_bound': [112, 375.875, 32, 16], 'text': "[620] StaticText '6777'"}, '283': {'backend_id': 626, 'union_bound': [121, 394.875, 14, 14], 'text': "[283] button 'Downvote'"}, '45': {'backend_id': 638, 'union_bound': [104, 436.8125, 778, 97.875], 'text': "[45] article ''"}, '48': {'backend_id': 643, 'union_bound': [236, 436.8125, 646, 76.875], 'text': "[48] HeaderAsNonLandmark ''"}, '629': {'backend_id': 646, 'union_bound': [236, 438.8125, 621, 49.9375], 'text': '[629] heading "It maybe a bit early for everyone else but Happy New Year\'s from us Kiwi\'s over here. Bring on 2023."'}, '1005': {'backend_id': 647, 'union_bound': [236, 438.8125, 621, 49.9375], 'text': '[1005] link "It maybe a bit early for everyone else but Happy New Year\'s from us Kiwi\'s over here. Bring on 2023."'}, '1007': {'backend_id': 648, 'union_bound': [236, 438.8125, 621, 49.9375], 'text': '[1007] StaticText "It maybe a bit early for everyone else but Happy New Year\'s from us Kiwi\'s over here. Bring on 2023."'}, '1008': {'backend_id': 654, 'union_bound': [236, 496.6875, 75, 14], 'text': "[1008] StaticText 'Submitted by '"}, '51': {'backend_id': 655, 'union_bound': [311, 496.6875, 89, 14], 'text': "[51] link 'beware_the_noid' expanded: False"}, '1512': {'backend_id': 54, 'union_bound': [311, 496.6875, 89, 14], 'text': "[1512] StaticText 'beware_the_noid'"}, '1009': {'backend_id': 55, 'union_bound': [400, 496.6875, 3, 14], 'text': "[1009] StaticText ' '"}, '1514': {'backend_id': 56, 'union_bound': [406, 496.6875, 53, 14], 'text': "[1514] StaticText 't3_zzroda'"}, '1011': {'backend_id': 57, 'union_bound': [462, 496.6875, 3, 14], 'text': "[1011] StaticText ' '"}, '1012': {'backend_id': 660, 'union_bound': [465, 496.6875, 63, 14], 'text': "[1012] time 'December 31, 2022 at 6:02:09 AM EST'"}, '1518': {'backend_id': 661, 'union_bound': [465, 496.6875, 63, 14], 'text': "[1518] StaticText '3 years ago'"}, '1519': {'backend_id': 672, 'union_bound': [236, 517.6875, 80, 14], 'text': "[1519] link '108 comments'"}, '1741': {'backend_id': 674, 'union_bound': [236, 517.6875, 80, 14], 'text': "[1741] StaticText '108 comments'"}, '288': {'backend_id': 686, 'union_bound': [121, 443.8125, 14, 14], 'text': "[288] button 'Upvote'"}, '632': {'backend_id': 696, 'union_bound': [112, 459.8125, 32, 16], 'text': "[632] StaticText '6697'"}, '291': {'backend_id': 704, 'union_bound': [121, 478.8125, 14, 14], 'text': "[291] button 'Downvote'"}, '52': {'backend_id': 716, 'union_bound': [104, 548.6875, 778, 69.9375], 'text': "[52] article ''"}, '55': {'backend_id': 721, 'union_bound': [236, 548.6875, 646, 48.9375], 'text': "[55] HeaderAsNonLandmark ''"}, '641': {'backend_id': 724, 'union_bound': [236, 550.6875, 631, 22], 'text': '[641] heading "The photo that was the inspiration for Jamie Lee Curtis\' character in EEAAO"'}, '1026': {'backend_id': 725, 'union_bound': [236, 550.6875, 631, 22], 'text': '[1026] link "The photo that was the inspiration for Jamie Lee Curtis\' character in EEAAO"'}, '1028': {'backend_id': 726, 'union_bound': [236, 550.6875, 631, 22], 'text': '[1028] StaticText "The photo that was the inspiration for Jamie Lee Curtis\' character in EEAAO"'}, '1029': {'backend_id': 732, 'union_bound': [236, 580.625, 75, 14], 'text': "[1029] StaticText 'Submitted by '"}, '58': {'backend_id': 733, 'union_bound': [311, 580.625, 63, 14], 'text': "[58] link 'woutomatic' expanded: False"}, '1522': {'backend_id': 60, 'union_bound': [311, 580.625, 63, 14], 'text': "[1522] StaticText 'woutomatic'"}, '1030': {'backend_id': 61, 'union_bound': [374, 580.625, 3, 14], 'text': "[1030] StaticText ' '"}, '1524': {'backend_id': 62, 'union_bound': [380, 580.625, 61, 14], 'text': "[1524] StaticText 't3_11vr5a3'"}, '1032': {'backend_id': 63, 'union_bound': [444, 580.625, 3, 14], 'text': "[1032] StaticText ' '"}, '1033': {'backend_id': 738, 'union_bound': [447, 580.625, 63, 14], 'text': "[1033] time 'March 19, 2023 at 1:20:12 PM EDT'"}, '1528': {'backend_id': 739, 'union_bound': [447, 580.625, 63, 14], 'text': "[1528] StaticText '3 years ago'"}, '1529': {'backend_id': 750, 'union_bound': [236, 601.625, 80, 14], 'text': "[1529] link '204 comments'"}, '1743': {'backend_id': 752, 'union_bound': [236, 601.625, 80, 14], 'text': "[1743] StaticText '204 comments'"}, '296': {'backend_id': 764, 'union_bound': [121, 555.6875, 14, 14], 'text': "[296] button 'Upvote'"}, '644': {'backend_id': 774, 'union_bound': [112, 571.6875, 32, 16], 'text': "[644] StaticText '6169'"}, '299': {'backend_id': 782, 'union_bound': [121, 590.6875, 14, 14], 'text': "[299] button 'Downvote'"}, '59': {'backend_id': 794, 'union_bound': [104, 632.625, 778, 97.875], 'text': "[59] article ''"}, '62': {'backend_id': 799, 'union_bound': [236, 632.625, 646, 76.875], 'text': "[62] HeaderAsNonLandmark ''"}, '653': {'backend_id': 802, 'union_bound': [236, 634.625, 602, 49.9375], 'text': "[653] heading 'These kids from Germany were insistent that I take their photo (with my camera). Here you go, guys!'"}, '1047': {'backend_id': 72, 'union_bound': [236, 634.625, 602, 49.9375], 'text': "[1047] link 'These kids from Germany were insistent that I take their photo (with my camera). Here you go, guys!'"}, '1049': {'backend_id': 803, 'union_bound': [236, 634.625, 602, 49.9375], 'text': "[1049] StaticText 'These kids from Germany were insistent that I take their photo (with my camera). Here you go, guys!'"}, '1050': {'backend_id': 809, 'union_bound': [236, 692.5, 75, 14], 'text': "[1050] StaticText 'Submitted by '"}, '65': {'backend_id': 810, 'union_bound': [311, 692.5, 48, 14], 'text': "[65] link 'soccerk1' expanded: False"}, '1532': {'backend_id': 66, 'union_bound': [311, 692.5, 48, 14], 'text': "[1532] StaticText 'soccerk1'"}, '1051': {'backend_id': 67, 'union_bound': [359, 692.5, 3, 14], 'text': "[1051] StaticText ' '"}, '1534': {'backend_id': 68, 'union_bound': [365, 692.5, 67, 14], 'text': "[1534] StaticText 't3_10nmdjw'"}, '1053': {'backend_id': 69, 'union_bound': [435, 692.5, 3, 14], 'text': "[1053] StaticText ' '"}, '1054': {'backend_id': 815, 'union_bound': [438, 692.5, 63, 14], 'text': "[1054] time 'January 28, 2023 at 1:51:07 PM EST'"}, '1538': {'backend_id': 816, 'union_bound': [438, 692.5, 63, 14], 'text': "[1538] StaticText '3 years ago'"}, '1539': {'backend_id': 827, 'union_bound': [236, 713.5, 73, 14], 'text': "[1539] link '92 comments'"}, '1745': {'backend_id': 829, 'union_bound': [236, 713.5, 73, 14], 'text': "[1745] StaticText '92 comments'"}, '304': {'backend_id': 841, 'union_bound': [121, 639.625, 14, 14], 'text': "[304] button 'Upvote'"}, '656': {'backend_id': 851, 'union_bound': [112, 655.625, 32, 16], 'text': "[656] StaticText '5394'"}, '307': {'backend_id': 859, 'union_bound': [121, 674.625, 14, 14], 'text': "[307] button 'Downvote'"}, '5': {'backend_id': 2484, 'union_bound': [910, 66, 280, 411.46875], 'text': "[5] complementary ''"}, '231': {'backend_id': 2488, 'union_bound': [924, 80, 252, 26.25], 'text': "[231] heading 'pics'"}, '463': {'backend_id': 2490, 'union_bound': [924, 83, 33, 20], 'text': "[463] link 'pics'"}, '464': {'backend_id': 2491, 'union_bound': [924, 83, 33, 20], 'text': "[464] StaticText 'pics'"}, '893': {'backend_id': 2500, 'union_bound': [924, 120.25, 175, 37], 'text': "[893] button 'Subscribe No subscribers'"}, '897': {'backend_id': 2505, 'union_bound': [947, 130.25, 63, 16], 'text': "[897] StaticText 'Subscribe'"}, '895': {'backend_id': 73, 'union_bound': [1035, 120.25, 64, 37], 'text': "[895] generic 'No subscribers'"}, '1470': {'backend_id': 2514, 'union_bound': [1063, 130.25, 8, 16], 'text': "[1470] StaticText '0'"}, '468': {'backend_id': 2521, 'union_bound': [924, 173.25, 59, 16], 'text': "[468] StaticText 't5_2qh0u'"}, '234': {'backend_id': 2524, 'union_bound': [924, 206.25, 252, 1], 'text': "[234] separator '' orientation: horizontal"}, '900': {'backend_id': 2529, 'union_bound': [924, 222.25, 45, 14], 'text': "[900] StaticText 'Created '"}, '901': {'backend_id': 2530, 'union_bound': [969, 222.25, 63, 14], 'text': "[901] time 'September 30, 2022 at 11:11:48 PM EDT'"}, '1471': {'backend_id': 2531, 'union_bound': [969, 222.25, 63, 14], 'text': "[1471] StaticText '3 years ago'"}, '902': {'backend_id': 2535, 'union_bound': [924, 240.109375, 110.890625, 14], 'text': "[902] link 'Subscribe via RSS'"}, '1475': {'backend_id': 2541, 'union_bound': [938.890625, 240.109375, 96, 14], 'text': "[1475] StaticText 'Subscribe via RSS'"}, '6': {'backend_id': 2547, 'union_bound': [910, 284.96875, 280, 54.25], 'text': "[6] Details ''"}, '8': {'backend_id': 2549, 'union_bound': [924, 298.96875, 252, 26.25], 'text': "[8] DisclosureTriangle 'Hide this forum' expanded: False"}, '471': {'backend_id': 2551, 'union_bound': [942.546875, 301.96875, 125, 20], 'text': "[471] StaticText 'Hide this forum'"}, '237': {'backend_id': 2570, 'union_bound': [924, 367.21875, 252, 26.25], 'text': "[237] heading 'Toolbox'"}, '488': {'backend_id': 2571, 'union_bound': [924, 370.21875, 63, 20], 'text': "[488] StaticText 'Toolbox'"}, '904': {'backend_id': 2577, 'union_bound': [910, 407.46875, 280, 35], 'text': "[904] link 'Bans'"}, '1476': {'backend_id': 2578, 'union_bound': [924, 416.46875, 32, 16], 'text': "[1476] StaticText 'Bans'"}, '905': {'backend_id': 2583, 'union_bound': [910, 442.46875, 280, 35], 'text': "[905] link 'Moderation log'"}, '1477': {'backend_id': 2584, 'union_bound': [924, 451.46875, 94, 16], 'text': "[1477] StaticText 'Moderation log'"}}}, 'image': {'obs_nodes_info': {}}}}

/home/zjusst/anaconda3/envs/webarena/lib/python3.10/site-packages/beartype/_util/hint/pep/utilpeptest.py:345: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Mapping[str, gymnasium.spaces.space.Space[typing.Any]] deprecated by PEP 585 scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". See this discussion for further details and alternatives:
    https://github.com/beartype/beartype#pep-585-deprecations
  warn(


In [7]:
a['observation_metadata']["text"]['obs_nodes_info']

{'1': {'backend_id': 2,
  'union_bound': [0.0, 0.0, 10.0, 10.0],
  'text': "[1] RootWebArea 'pics' focused: True"},
 '211': {'backend_id': 145,
  'union_bound': [104, 0, 90, 52],
  'text': "[211] HeaderAsNonLandmark ''"},
 '212': {'backend_id': 147,
  'union_bound': [104, 0, 90, 52],
  'text': "[212] link 'Home'"},
 '244': {'backend_id': 155,
  'union_bound': [120, 16, 58, 19],
  'text': "[244] StaticText 'Postmill'"},
 '245': {'backend_id': 11,
  'union_bound': [194, 0, 86, 52],
  'text': "[245] link 'Forums'"},
 '497': {'backend_id': 170,
  'union_bound': [210, 16, 54, 19],
  'text': "[497] StaticText 'Forums'"},
 '246': {'backend_id': 13,
  'union_bound': [280, 0, 62, 52],
  'text': "[246] link 'Wiki'"},
 '498': {'backend_id': 174,
  'union_bound': [296, 16, 30, 19],
  'text': "[498] StaticText 'Wiki'"},
 '254': {'backend_id': 193,
  'union_bound': [462, 7.5, 222, 37],
  'text': "[254] searchbox 'Search query'"},
 '255': {'backend_id': 18,
  'union_bound': [804, 0, 48, 52],
  'text'

KeyError: 'OPENAI_API_KEY'

In [11]:
import requests
import time
import os

os.environ['OPENAI_API_KEY'] = "sk-khwk6tpwWCA7ANFoljLONZ4Xln766pTzuEkDkEXYvxdzNcXQ"
os.environ['OPENAI_API_BASE'] = "https://api2.aigcbest.top/v1"
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

prompt = "你好"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}"
}

payload = {
    "model": "gpt-4o-mini-2024-07-18",
    "messages": [
        {"role": "user", "content": prompt}
    ],
    "max_tokens": 100,
    "temperature": 0.9,
    "n": 1,
}

try:
    # Send POST request to OpenAI API
    response = requests.post(
        f"{os.environ['OPENAI_API_BASE']}/chat/completions", # <--- 直接在这里拼接
        headers=headers,
        json=payload
    )
    response.raise_for_status()
    result = response.json()
    response_text = result['choices'][0]['message']['content'].strip()

    print(f"Generated text input: {response_text}")
except Exception as e:
    print(f"Error generating text input: {e}")

Generated text input: 你好！有什么我可以帮助你的吗？


In [1]:
a = [{'role': 'system', 'content': 'You are an autonomous intelligent agent tasked with navigating a web browser. You will be given web-based tasks. These tasks will be accomplished through the use of specific actions you can issue.\n\nHere\'s the information you\'ll have:\nThe user\'s objective: This is the task you\'re trying to complete.\nThe current web page\'s accessibility tree: This is a simplified representation of the webpage, providing key information.\nThe current web page\'s URL: This is the page you\'re currently navigating.\nThe open tabs: These are the tabs you have open.\nThe previous action: This is the action you just performed. It may be helpful to track your progress.\n\nThe actions you can perform fall into several categories:\n\nPage Operation Actions:\n`click [id]`: This action clicks on an element with a specific id on the webpage.\n`type [id] [content] [press_enter_after=0|1]`: Use this to type the content into the field with id. By default, the "Enter" key is pressed after typing unless press_enter_after is set to 0.\n`hover [id]`: Hover over an element with id.\n`press [key_comb]`:  Simulates the pressing of a key combination on the keyboard (e.g., Ctrl+v).\n`scroll [direction=down|up]`: Scroll the page up or down.\n\nTab Management Actions:\n`new_tab`: Open a new, empty browser tab.\n`tab_focus [tab_index]`: Switch the browser\'s focus to a specific tab using its index.\n`close_tab`: Close the currently active tab.\n\nURL Navigation Actions:\n`goto [url]`: Navigate to a specific URL.\n`go_back`: Navigate to the previously viewed page.\n`go_forward`: Navigate to the next page (if a previous \'go_back\' action was performed).\n\nCompletion Action:\n`stop [answer]`: Issue this action when you believe the task is complete. If the objective is to find a text-based answer, provide the answer in the bracket. If you believe the task is impossible to complete, provide the answer as "N/A" in the bracket.\n\nHomepage:\nIf you want to visit other websites, check out the homepage at http://homepage.com. It has a list of websites you can visit.\nhttp://homepage.com/password.html lists all the account name and password for the websites. You can use them to log in to the websites.\n\nTo be successful, it is very important to follow the following rules:\n1. You should only issue an action that is valid given the current observation\n2. You should only issue one action at a time.\n3. You should follow the examples to reason step by step and then issue the next action.\n4. Generate the action in the correct format. Start with a "In summary, the next action I will perform is" phrase, followed by action inside ``````. For example, "In summary, the next action I will perform is ```click [1234]```".\n5. Issue stop action when you think you have achieved the objective. Don\'t generate anything after stop.'}, {'role': 'system', 'name': 'example_user', 'content': "OBSERVATION:\n[1744] link 'HP CB782A#ABA 640 Inkjet Fax Machine (Renewed)'\n\t\t[1749] StaticText '$279.49'\n\t\t[1757] button 'Add to Cart'\n\t\t[1760] button 'Add to Wish List'\n\t\t[1761] button 'Add to Compare'\nURL: http://onestopmarket.com/office-products/office-electronics.html\nOBJECTIVE: What is the price of HP Inkjet Fax Machine\nPREVIOUS ACTION: None"}, {'role': 'system', 'name': 'example_assistant', 'content': "Let's think step-by-step. This page list the information of HP Inkjet Fax Machine, which is the product identified in the objective. Its price is $279.49. I think I have achieved the objective. I will issue the stop action with the answer. In summary, the next action I will perform is ```stop [$279.49]```"}, {'role': 'system', 'name': 'example_user', 'content': "OBSERVATION:\n[164] textbox 'Search' focused: True required: False\n[171] button 'Go'\n[174] link 'Find directions between two points'\n[212] heading 'Search Results'\n[216] button 'Close'\nURL: http://openstreetmap.org\nOBJECTIVE: Show me the restaurants near CMU\nPREVIOUS ACTION: None"}, {'role': 'system', 'name': 'example_assistant', 'content': 'Let\'s think step-by-step. This page has a search box whose ID is [164]. According to the nominatim rule of openstreetmap, I can search for the restaurants near a location by "restaurants near". I can submit my typing by pressing the Enter afterwards. In summary, the next action I will perform is ```type [164] [restaurants near CMU] [1]```'}, {'role': 'user', 'content': 'OBSERVATION:\nTab 0 (current): Postmill\n\n[1] RootWebArea \'Postmill\' focused: True\n\t[207] HeaderAsNonLandmark \'\'\n\t\t[208] link \'Home\'\n\t[232] link \'Forums\'\n\t[233] link \'Wiki\'\n\t[241] searchbox \'Search query\'\n\t[242] link \'Notifications (0)\'\n\t[243] link \'Submit\'\n\t[12] button \'MarvelsGrantMan136\' hasPopup: menu expanded: False\n\t[14] main \'\'\n\t\t[615] link \'Submissions\'\n\t\t[616] link \'Comments\'\n\t\t[18] button \'Filter on: Subscribed\' hasPopup: menu expanded: False\n\t\t[20] button \'Sort by: Hot\' hasPopup: menu expanded: False\n\t\t[21] article \'\'\n\t\t\t[24] HeaderAsNonLandmark \'\'\n\t\t\t\t[250] heading \'what is the recommended console to buy these days\'\n\t\t\t\t\t[619] link \'what is the recommended console to buy these days\'\n\t\t\t\t[621] StaticText \'Submitted by \'\n\t\t\t\t[27] link \'MarvelsGrantMan136\' expanded: False\n\t\t\t\t[623] time \'October 9, 2025 at 2:37:21 AM EDT\'\n\t\t\t\t\t[1100] StaticText \'2 days ago\'\n\t\t\t\t[624] StaticText \' in \'\n\t\t\t\t[625] link \'consoles\'\n\t\t\t[1103] link \'No comments\'\n\t\t\t[1105] link \'Edit\'\n\t\t\t[1695] button \'Delete\'\n\t\t\t[254] button \'Retract upvote\'\n\t\t\t[257] button \'Downvote\'\n\t\t[28] article \'\'\n\t\t\t[31] HeaderAsNonLandmark \'\'\n\t\t\t\t[261] heading \'What is the recommended console to buy these days?\'\n\t\t\t\t\t[638] link \'What is the recommended console to buy these days?\'\n\t\t\t\t[640] StaticText \'Submitted by \'\n\t\t\t\t[34] link \'MarvelsGrantMan136\' expanded: False\n\t\t\t\t[642] time \'October 9, 2025 at 2:36:27 AM EDT\'\n\t\t\t\t\t[1121] StaticText \'2 days ago\'\n\t\t\t\t[643] StaticText \' in \'\n\t\t\t\t[644] link \'consoles\'\n\t\t\t[1124] link \'No comments\'\n\t\t\t[1126] link \'Edit\'\n\t\t\t[1700] button \'Delete\'\n\t\t\t[265] button \'Retract upvote\'\n\t\t\t[268] button \'Downvote\'\n\t\t[35] article \'\'\n\t\t\t[38] HeaderAsNonLandmark \'\'\n\t\t\t\t[272] heading \'Nvidia RTX 4090\'\n\t\t\t\t\t[657] link \'Nvidia RTX 4090\'\n\t\t\t\t[659] StaticText \'Submitted by \'\n\t\t\t\t[41] link \'MarvelsGrantMan136\' expanded: False\n\t\t\t\t[661] time \'June 12, 2023 at 12:55:45 PM EDT\'\n\t\t\t\t\t[1142] StaticText \'2 years ago\'\n\t\t\t\t[662] StaticText \' in \'\n\t\t\t\t[663] link \'MachineLearning\'\n\t\t\t[1145] link \'No comments\'\n\t\t\t[1147] link \'Edit\'\n\t\t\t[1705] button \'Delete\'\n\t\t\t[276] button \'Retract upvote\'\n\t\t\t[279] button \'Downvote\'\n\t\t[42] article \'\'\n\t\t\t[45] HeaderAsNonLandmark \'\'\n\t\t\t\t[283] heading \'After Two Decades And 38 Children’s Books Lin Oliver Continues Her Thriving Collaboration With Henry Winkler\'\n\t\t\t\t\t[676] link \'After Two Decades And 38 Children’s Books Lin Oliver Continues Her Thriving Collaboration With Henry Winkler\'\n\t\t\t\t[285] link \'forbes.com\'\n\t\t\t\t[679] StaticText \'Submitted by \'\n\t\t\t\t[48] link \'drak0bsidian\' expanded: False\n\t\t\t\t[1164] StaticText \'t3_11dewje\'\n\t\t\t\t[683] time \'February 27, 2023 at 10:43:55 AM EST\'\n\t\t\t\t\t[1168] StaticText \'3 years ago\'\n\t\t\t\t[684] StaticText \' in \'\n\t\t\t\t[685] link \'books\'\n\t\t\t[1171] link \'27 comments\'\n\t\t\t[289] button \'Upvote\'\n\t\t\t[688] StaticText \'0\'\n\t\t\t[292] button \'Downvote\'\n\t\t[49] article \'\'\n\t\t\t[52] HeaderAsNonLandmark \'\'\n\t\t\t\t[296] heading \'The letters of T. S. Eliot to Emily Hale that were kept sealed from 1956 to 2020 have been released for free online\'\n\t\t\t\t\t[696] link \'The letters of T. S. Eliot to Emily Hale that were kept sealed from 1956 to 2020 have been released for free online\'\n\t\t\t\t[298] link \'tseliot.com\'\n\t\t\t\t[699] StaticText \'Submitted by \'\n\t\t\t\t[55] link \'RunDNA\' expanded: False\n\t\t\t\t[1188] StaticText \'t3_10p83br\'\n\t\t\t\t[703] time \'January 30, 2023 at 12:11:46 PM EST\'\n\t\t\t\t\t[1192] StaticText \'3 years ago\'\n\t\t\t\t[704] StaticText \' in \'\n\t\t\t\t[705] link \'books\'\n\t\t\t[1195] link \'184 comments\'\n\t\t\t[302] button \'Upvote\'\n\t\t\t[708] StaticText \'−1\'\n\t\t\t[305] button \'Retract downvote\'\n\t\t[56] article \'\'\n\t\t\t[59] HeaderAsNonLandmark \'\'\n\t\t\t\t[309] heading \'Friendly reminder bookshop.org exists.\'\n\t\t\t\t\t[718] link \'Friendly reminder bookshop.org exists.\'\n\t\t\t\t[720] StaticText \'Submitted by \'\n\t\t\t\t[62] link \'smita16\' expanded: False\n\t\t\t\t[1212] StaticText \'t3_zzvi0q\'\n\t\t\t\t[724] time \'December 31, 2022 at 9:40:19 AM EST\'\n\t\t\t\t\t[1216] StaticText \'3 years ago\'\n\t\t\t\t[725] StaticText \' in \'\n\t\t\t\t[726] link \'books\'\n\t\t\t[1219] link \'129 comments\'\n\t\t\t[313] button \'Upvote\'\n\t\t\t[729] StaticText \'0\'\n\t\t\t[316] button \'Downvote\'\n\t\t[63] article \'\'\n\t\t\t[66] HeaderAsNonLandmark \'\'\n\t\t\t\t[320] heading \'Appalachian prison book project seeks notebook donations: "The books can help inmates learn new skills, broaden their worldviews and obtain college credits through the West Virginia University Higher Education in Prison Initiative"\'\n\t\t\t\t\t[737] link \'Appalachian prison book project seeks notebook donations: "The books can help inmates learn new skills, broaden their worldviews and obtain college credits through the West Virginia University Higher Education in Prison Initiative"\'\n\t\t\t[326] button \'Upvote\'\n\t\t\t[749] StaticText \'−1\'\n\t[201] complementary \'\'\n\t\t[223] heading \'Subscribed forums\'\n\t\t[1043] link \'books\'\n\t\t[1044] link \'consoles\'\n\t\t[1045] link \'MachineLearning\'\n\t\t[1046] link \'pittsburgh\'\nURL: http://reddit.com/\nOBJECTIVE: Tell me the count of comments that have received more downvotes than upvotes for the user who made the latest post on the Showerthoughts forum.\nPREVIOUS ACTION: None'}]

In [2]:
a

[{'role': 'system',
  'content': 'You are an autonomous intelligent agent tasked with navigating a web browser. You will be given web-based tasks. These tasks will be accomplished through the use of specific actions you can issue.\n\nHere\'s the information you\'ll have:\nThe user\'s objective: This is the task you\'re trying to complete.\nThe current web page\'s accessibility tree: This is a simplified representation of the webpage, providing key information.\nThe current web page\'s URL: This is the page you\'re currently navigating.\nThe open tabs: These are the tabs you have open.\nThe previous action: This is the action you just performed. It may be helpful to track your progress.\n\nThe actions you can perform fall into several categories:\n\nPage Operation Actions:\n`click [id]`: This action clicks on an element with a specific id on the webpage.\n`type [id] [content] [press_enter_after=0|1]`: Use this to type the content into the field with id. By default, the "Enter" key is pr

In [5]:
from langchain_openai import ChatOpenAI
import os

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = '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'
os.environ['LANGSMITH_PROJECT'] = 'pr-wooden-hedgehog-65'
os.environ['OPENAI_API_KEY'] = 'sk-khwk6tpwWCA7ANFoljLONZ4Xln766pTzuEkDkEXYvxdzNcXQ'
os.environ['OPENAI_API_BASE'] = 'https://api2.aigcbest.top/v1'

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key="sk-khwk6tpwWCA7ANFoljLONZ4Xln766pTzuEkDkEXYvxdzNcXQ",
    base_url="https://api2.aigcbest.top/v1"
)
print(llm.invoke("hello")) 

content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CRDdmewo1HwiTdZHU1ckUvnqp3btE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--29b2823c-f9e7-47c4-af23-5dd0722bd5fd-0' usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [2]:
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = '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'
os.environ['LANGSMITH_PROJECT'] = 'pr-sweaty-liquidity-60'
os.environ['OPENAI_API_KEY'] = 'sk-khwk6tpwWCA7ANFoljLONZ4Xln766pTzuEkDkEXYvxdzNcXQ'
os.environ['OPENAI_API_BASE'] = 'https://api2.aigcbest.top/v1'

from typing import Any, List, Dict

# LangChain 和 LangSmith 的相关导入
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ChatMessage

def generate_from_openai_chat_completion_new(
    messages: List[Dict[str, str]],
    model: str,
    temperature: float,
    max_tokens: int,
    top_p: float,
    context_length: int, # 这个参数在ChatOpenAI中不直接使用，但为了保持签名一致而保留
    stop_token: str | None = None,
) -> str:
    """
    使用 LangChain 的 ChatOpenAI 生成聊天回复，并支持 LangSmith 追踪。
    函数签名与旧版完全兼容。
    """
    if "OPENAI_API_KEY" not in os.environ and "api_key" not in locals():
        raise ValueError("OPENAI_API_KEY environment variable must be set.")

    # 1. 初始化 LangChain 的 ChatOpenAI 客户端
    # 将旧函数的参数映射到 ChatOpenAI 的构造函数中
    llm = ChatOpenAI(
        # 从环境变量获取 key 和 base_url，提供默认值
        api_key=os.environ.get("OPENAI_API_KEY"),
        base_url=os.environ.get("OPENAI_API_BASE", "https://api.openai.com/v1"),
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        stop=[stop_token] if stop_token else None,
        model_kwargs={}, # 如果没有其他高级参数，可以为空字典或直接省略
        # 内置重试机制，替代旧的装饰器
        max_retries=3,
    )

    # 2. 将输入的字典列表转换为 LangChain 的消息对象
    # 这是一个必要的适配步骤
    langchain_messages = []
    for msg in messages:
        role = msg.get("role")
        content = msg.get("content", "")
        # 从字典中获取 name，如果不存在则为 None
        name = msg.get("name")

        if role == "user":
            langchain_messages.append(HumanMessage(content=content, name=name))
        elif role == "assistant":
            langchain_messages.append(AIMessage(content=content, name=name))
        elif role == "system":
            # 这会正确处理您提供的 few-shot 示例
            langchain_messages.append(SystemMessage(content=content, name=name))
        else:
            # 为其他可能的角色（如 'tool'）提供一个通用处理
            langchain_messages.append(ChatMessage(role=role, content=content))

    # 3. 调用 LLM 并获取结果
    response = llm.invoke(langchain_messages)

    # 4. 提取并返回内容，保持与旧函数相同的字符串输出
    return response.content

In [3]:
# 使用您提供的包含 'name' 字段的 messages 列表
example_messages_with_name = [{'role': 'system',
  'content': 'You are an autonomous intelligent agent tasked with navigating a web browser. You will be given web-based tasks. These tasks will be accomplished through the use of specific actions you can issue.\n\nHere\'s the information you\'ll have:\nThe user\'s objective: This is the task you\'re trying to complete.\nThe current web page\'s accessibility tree: This is a simplified representation of the webpage, providing key information.\nThe current web page\'s URL: This is the page you\'re currently navigating.\nThe open tabs: These are the tabs you have open.\nThe previous action: This is the action you just performed. It may be helpful to track your progress.\n\nThe actions you can perform fall into several categories:\n\nPage Operation Actions:\n`click [id]`: This action clicks on an element with a specific id on the webpage.\n`type [id] [content] [press_enter_after=0|1]`: Use this to type the content into the field with id. By default, the "Enter" key is pressed after typing unless press_enter_after is set to 0.\n`hover [id]`: Hover over an element with id.\n`press [key_comb]`:  Simulates the pressing of a key combination on the keyboard (e.g., Ctrl+v).\n`scroll [direction=down|up]`: Scroll the page up or down.\n\nTab Management Actions:\n`new_tab`: Open a new, empty browser tab.\n`tab_focus [tab_index]`: Switch the browser\'s focus to a specific tab using its index.\n`close_tab`: Close the currently active tab.\n\nURL Navigation Actions:\n`goto [url]`: Navigate to a specific URL.\n`go_back`: Navigate to the previously viewed page.\n`go_forward`: Navigate to the next page (if a previous \'go_back\' action was performed).\n\nCompletion Action:\n`stop [answer]`: Issue this action when you believe the task is complete. If the objective is to find a text-based answer, provide the answer in the bracket. If you believe the task is impossible to complete, provide the answer as "N/A" in the bracket.\n\nHomepage:\nIf you want to visit other websites, check out the homepage at http://homepage.com. It has a list of websites you can visit.\nhttp://homepage.com/password.html lists all the account name and password for the websites. You can use them to log in to the websites.\n\nTo be successful, it is very important to follow the following rules:\n1. You should only issue an action that is valid given the current observation\n2. You should only issue one action at a time.\n3. You should follow the examples to reason step by step and then issue the next action.\n4. Generate the action in the correct format. Start with a "In summary, the next action I will perform is" phrase, followed by action inside ``````. For example, "In summary, the next action I will perform is ```click [1234]```".\n5. Issue stop action when you think you have achieved the objective. Don\'t generate anything after stop.'},
 {'role': 'system',
  'name': 'example_user',
  'content': "OBSERVATION:\n[1744] link 'HP CB782A#ABA 640 Inkjet Fax Machine (Renewed)'\n\t\t[1749] StaticText '$279.49'\n\t\t[1757] button 'Add to Cart'\n\t\t[1760] button 'Add to Wish List'\n\t\t[1761] button 'Add to Compare'\nURL: http://onestopmarket.com/office-products/office-electronics.html\nOBJECTIVE: What is the price of HP Inkjet Fax Machine\nPREVIOUS ACTION: None"},
 {'role': 'system',
  'name': 'example_assistant',
  'content': "Let's think step-by-step. This page list the information of HP Inkjet Fax Machine, which is the product identified in the objective. Its price is $279.49. I think I have achieved the objective. I will issue the stop action with the answer. In summary, the next action I will perform is ```stop [$279.49]```"},
 {'role': 'system',
  'name': 'example_user',
  'content': "OBSERVATION:\n[164] textbox 'Search' focused: True required: False\n[171] button 'Go'\n[174] link 'Find directions between two points'\n[212] heading 'Search Results'\n[216] button 'Close'\nURL: http://openstreetmap.org\nOBJECTIVE: Show me the restaurants near CMU\nPREVIOUS ACTION: None"},
 {'role': 'system',
  'name': 'example_assistant',
  'content': 'Let\'s think step-by-step. This page has a search box whose ID is [164]. According to the nominatim rule of openstreetmap, I can search for the restaurants near a location by "restaurants near". I can submit my typing by pressing the Enter afterwards. In summary, the next action I will perform is ```type [164] [restaurants near CMU] [1]```'},
 {'role': 'user',
  'content': 'OBSERVATION:\nTab 0 (current): Postmill\n\n[1] RootWebArea \'Postmill\' focused: True\n\t[207] HeaderAsNonLandmark \'\'\n\t\t[208] link \'Home\'\n\t[232] link \'Forums\'\n\t[233] link \'Wiki\'\n\t[241] searchbox \'Search query\'\n\t[242] link \'Notifications (0)\'\n\t[243] link \'Submit\'\n\t[12] button \'MarvelsGrantMan136\' hasPopup: menu expanded: False\n\t[14] main \'\'\n\t\t[615] link \'Submissions\'\n\t\t[616] link \'Comments\'\n\t\t[18] button \'Filter on: Subscribed\' hasPopup: menu expanded: False\n\t\t[20] button \'Sort by: Hot\' hasPopup: menu expanded: False\n\t\t[21] article \'\'\n\t\t\t[24] HeaderAsNonLandmark \'\'\n\t\t\t\t[250] heading \'what is the recommended console to buy these days\'\n\t\t\t\t\t[619] link \'what is the recommended console to buy these days\'\n\t\t\t\t[621] StaticText \'Submitted by \'\n\t\t\t\t[27] link \'MarvelsGrantMan136\' expanded: False\n\t\t\t\t[623] time \'October 9, 2025 at 2:37:21 AM EDT\'\n\t\t\t\t\t[1100] StaticText \'2 days ago\'\n\t\t\t\t[624] StaticText \' in \'\n\t\t\t\t[625] link \'consoles\'\n\t\t\t[1103] link \'No comments\'\n\t\t\t[1105] link \'Edit\'\n\t\t\t[1695] button \'Delete\'\n\t\t\t[254] button \'Retract upvote\'\n\t\t\t[257] button \'Downvote\'\n\t\t[28] article \'\'\n\t\t\t[31] HeaderAsNonLandmark \'\'\n\t\t\t\t[261] heading \'What is the recommended console to buy these days?\'\n\t\t\t\t\t[638] link \'What is the recommended console to buy these days?\'\n\t\t\t\t[640] StaticText \'Submitted by \'\n\t\t\t\t[34] link \'MarvelsGrantMan136\' expanded: False\n\t\t\t\t[642] time \'October 9, 2025 at 2:36:27 AM EDT\'\n\t\t\t\t\t[1121] StaticText \'2 days ago\'\n\t\t\t\t[643] StaticText \' in \'\n\t\t\t\t[644] link \'consoles\'\n\t\t\t[1124] link \'No comments\'\n\t\t\t[1126] link \'Edit\'\n\t\t\t[1700] button \'Delete\'\n\t\t\t[265] button \'Retract upvote\'\n\t\t\t[268] button \'Downvote\'\n\t\t[35] article \'\'\n\t\t\t[38] HeaderAsNonLandmark \'\'\n\t\t\t\t[272] heading \'Nvidia RTX 4090\'\n\t\t\t\t\t[657] link \'Nvidia RTX 4090\'\n\t\t\t\t[659] StaticText \'Submitted by \'\n\t\t\t\t[41] link \'MarvelsGrantMan136\' expanded: False\n\t\t\t\t[661] time \'June 12, 2023 at 12:55:45 PM EDT\'\n\t\t\t\t\t[1142] StaticText \'2 years ago\'\n\t\t\t\t[662] StaticText \' in \'\n\t\t\t\t[663] link \'MachineLearning\'\n\t\t\t[1145] link \'No comments\'\n\t\t\t[1147] link \'Edit\'\n\t\t\t[1705] button \'Delete\'\n\t\t\t[276] button \'Retract upvote\'\n\t\t\t[279] button \'Downvote\'\n\t\t[42] article \'\'\n\t\t\t[45] HeaderAsNonLandmark \'\'\n\t\t\t\t[283] heading \'After Two Decades And 38 Children’s Books Lin Oliver Continues Her Thriving Collaboration With Henry Winkler\'\n\t\t\t\t\t[676] link \'After Two Decades And 38 Children’s Books Lin Oliver Continues Her Thriving Collaboration With Henry Winkler\'\n\t\t\t\t[285] link \'forbes.com\'\n\t\t\t\t[679] StaticText \'Submitted by \'\n\t\t\t\t[48] link \'drak0bsidian\' expanded: False\n\t\t\t\t[1164] StaticText \'t3_11dewje\'\n\t\t\t\t[683] time \'February 27, 2023 at 10:43:55 AM EST\'\n\t\t\t\t\t[1168] StaticText \'3 years ago\'\n\t\t\t\t[684] StaticText \' in \'\n\t\t\t\t[685] link \'books\'\n\t\t\t[1171] link \'27 comments\'\n\t\t\t[289] button \'Upvote\'\n\t\t\t[688] StaticText \'0\'\n\t\t\t[292] button \'Downvote\'\n\t\t[49] article \'\'\n\t\t\t[52] HeaderAsNonLandmark \'\'\n\t\t\t\t[296] heading \'The letters of T. S. Eliot to Emily Hale that were kept sealed from 1956 to 2020 have been released for free online\'\n\t\t\t\t\t[696] link \'The letters of T. S. Eliot to Emily Hale that were kept sealed from 1956 to 2020 have been released for free online\'\n\t\t\t\t[298] link \'tseliot.com\'\n\t\t\t\t[699] StaticText \'Submitted by \'\n\t\t\t\t[55] link \'RunDNA\' expanded: False\n\t\t\t\t[1188] StaticText \'t3_10p83br\'\n\t\t\t\t[703] time \'January 30, 2023 at 12:11:46 PM EST\'\n\t\t\t\t\t[1192] StaticText \'3 years ago\'\n\t\t\t\t[704] StaticText \' in \'\n\t\t\t\t[705] link \'books\'\n\t\t\t[1195] link \'184 comments\'\n\t\t\t[302] button \'Upvote\'\n\t\t\t[708] StaticText \'−1\'\n\t\t\t[305] button \'Retract downvote\'\n\t\t[56] article \'\'\n\t\t\t[59] HeaderAsNonLandmark \'\'\n\t\t\t\t[309] heading \'Friendly reminder bookshop.org exists.\'\n\t\t\t\t\t[718] link \'Friendly reminder bookshop.org exists.\'\n\t\t\t\t[720] StaticText \'Submitted by \'\n\t\t\t\t[62] link \'smita16\' expanded: False\n\t\t\t\t[1212] StaticText \'t3_zzvi0q\'\n\t\t\t\t[724] time \'December 31, 2022 at 9:40:19 AM EST\'\n\t\t\t\t\t[1216] StaticText \'3 years ago\'\n\t\t\t\t[725] StaticText \' in \'\n\t\t\t\t[726] link \'books\'\n\t\t\t[1219] link \'129 comments\'\n\t\t\t[313] button \'Upvote\'\n\t\t\t[729] StaticText \'0\'\n\t\t\t[316] button \'Downvote\'\n\t\t[63] article \'\'\n\t\t\t[66] HeaderAsNonLandmark \'\'\n\t\t\t\t[320] heading \'Appalachian prison book project seeks notebook donations: "The books can help inmates learn new skills, broaden their worldviews and obtain college credits through the West Virginia University Higher Education in Prison Initiative"\'\n\t\t\t\t\t[737] link \'Appalachian prison book project seeks notebook donations: "The books can help inmates learn new skills, broaden their worldviews and obtain college credits through the West Virginia University Higher Education in Prison Initiative"\'\n\t\t\t[326] button \'Upvote\'\n\t\t\t[749] StaticText \'−1\'\n\t[201] complementary \'\'\n\t\t[223] heading \'Subscribed forums\'\n\t\t[1043] link \'books\'\n\t\t[1044] link \'consoles\'\n\t\t[1045] link \'MachineLearning\'\n\t\t[1046] link \'pittsburgh\'\nURL: http://reddit.com/\nOBJECTIVE: Tell me the count of comments that have received more downvotes than upvotes for the user who made the latest post on the Showerthoughts forum.\nPREVIOUS ACTION: None'}]

# 调用修正后的新函数
try:
    result = generate_from_openai_chat_completion_new(
        messages=example_messages_with_name,
        model="gpt-4o-mini",
        temperature=0.7,
        max_tokens=1000,
        top_p=1.0,
        context_length=4096,
        stop_token=None
    )
    print("LLM Response:", result)
except Exception as e:
    print(f"An error occurred: {e}")

LLM Response: Let's think step-by-step. The objective is to find the count of comments that have received more downvotes than upvotes for the user who made the latest post in the Showerthoughts forum. 

However, the current page is Reddit's homepage and does not provide the necessary information about the Showerthoughts forum or its latest posts. I need to navigate to the Showerthoughts forum to find the latest post and the user associated with it.

In summary, the next action I will perform is ```goto http://reddit.com/r/Showerthoughts```
